In [1]:
# 1. 导入依赖 & 读取数据
import pandas as pd

# 载入明细和标签，字段名与 featureprocessing-Copy1.ipynb 保持一致
df = pd.read_csv('model1_data.csv', encoding='gbk', low_memory=False)
dflabel = pd.read_csv('model1_label.csv', encoding='gbk')

# 重命名字段并合并标签
df.rename(columns={
    '卡号': 'card_id',
    '机构名称': 'org_id',
    '结算日期时间': 'settle_time',
    '明细项目交易费用': 'fee',
}, inplace=True)

dflabel.rename(columns={'卡号': 'card_id', '标签': 'label'}, inplace=True)

df = pd.merge(df, dflabel, on='card_id', how='inner')

print(df[['card_id', '明细项目名称']].head())

                                card_id       明细项目名称
0  ff88846b-56ec-4d2f-a2fc-aca3c116c865        尼可地尔片
1  ff88846b-56ec-4d2f-a2fc-aca3c116c865  头孢克洛缓释片(II)
2  ff88846b-56ec-4d2f-a2fc-aca3c116c865      乳果糖口服溶液
3  ff88846b-56ec-4d2f-a2fc-aca3c116c865        艾司唑仑片
4  dccc6fc4-d367-420f-846d-ef5ece5cc1d2    盐酸地尔硫卓缓释片


In [2]:
# 2. 构造账户级“项目文档”并计算 TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# 去除缺失项目名称，只保留 card_id 和 明细项目名称
df_items = df.dropna(subset=['明细项目名称'])[['card_id', '明细项目名称']]

# 按账户聚合为项目列表
card_items = (
    df_items
    .groupby('card_id')['明细项目名称']
    .apply(list)
)

# 构造“文档字符串”：每个账户的项目序列，用空格连接
card_docs = card_items.apply(lambda items: ' '.join(map(str, items)))

print('账户数:', len(card_docs))
print(card_docs.head())

# 以完整项目名称为 token，不拆分中文
vectorizer = TfidfVectorizer(
    analyzer='word',
    token_pattern=r'[^ ]+',  # 按空格分词
    min_df=2,                # 至少在 2 个账户中出现
)

tfidf_matrix = vectorizer.fit_transform(card_docs.values)
print('TF-IDF 形状:', tfidf_matrix.shape)

card_ids = card_docs.index.to_list()

账户数: 8917
card_id
00022092-02fc-45e0-83f2-c51a0d02f2d0    赤小豆 芡实 白茯苓 人参片 薏苡仁 百乐眠胶囊 拉坦前列素滴眼液 马来酸噻吗洛尔滴眼液 芪...
000e9b7e-6a96-4eda-947b-425e964e1212    甲磺酸多沙唑嗪缓释片 宁泌泰胶囊 氯化钠注射液 丹红注射液 氯化钠注射液 盐酸倍他司汀注射液...
000f8286-aa23-42d7-8510-2fab100bcc7b    硝苯地平控释片 胞磷胆碱钠片 盐酸舍曲林片 胞磷胆碱钠片 丁丙诺啡透皮贴剂 金水宝片 银杏叶...
00117f6c-e739-4913-b453-85a118a47123    宣肺止嗽合剂 左氧氟沙星片 复方丹参滴丸 迈之灵片 头孢克洛缓释片 吡格列酮二甲双胍片 消渴...
001c5c03-1db7-4303-934e-21decf219ab1    参松养心胶囊 麝香保心丸 利伐沙班片 维生素B2片 双歧杆菌三联活菌胶囊 胰激肽原酶肠溶片 ...
Name: 明细项目名称, dtype: object
TF-IDF 形状: (8917, 3385)


In [6]:
# 3. 基于 TF-IDF 向量训练 Isolation Forest
from sklearn.ensemble import IsolationForest

# 无监督异常检测模型
iso = IsolationForest(
    n_estimators=200,
    max_samples='auto',
    contamination=0.2,  # 预期异常比例，可根据业务调整
    random_state=42,
    n_jobs=-1,
)

iso.fit(tfidf_matrix)

# 预测：-1 为异常，1 为正常
pred_labels = iso.predict(tfidf_matrix)
# 决策函数：值越小越异常
anomaly_scores = iso.decision_function(tfidf_matrix)

print('预测标签分布:', pd.Series(pred_labels).value_counts())

预测标签分布:  1    7133
-1    1784
Name: count, dtype: int64


In [7]:
# 4. 结果整理与导出

result_df = pd.DataFrame({
    'card_id': card_ids,
    'iso_label': pred_labels,       # -1 异常，1 正常
    'iso_score': anomaly_scores,    # 越小越异常
})

# 按异常分数从小到大排序（最异常的在最上面）
result_df = result_df.sort_values('iso_score')

# 保存结果
result_df.to_csv('isolation_forest_tfidf_results.csv', index=False, encoding='utf-8')

result_df.head(10)

,card_id,iso_label,iso_score
203,05f74fe7-065e-4848-86f9-676b95e524bf,-1,-0.047669
3578,68b531a1-b5b2-421c-8792-7230036c158a,-1,-0.045986
948,1adafee7-442d-4615-a2b6-4485c5119682,-1,-0.040930
5794,a7f1124f-f599-49f1-9b13-90d344d009d8,-1,-0.040558
3009,59611870-cf0c-4931-b462-588ccebeeded,-1,-0.039567
8739,fa932c11-47fe-47b3-85e2-3f86562a9bd2,-1,-0.036467
1132,2111d431-6770-414d-8996-9c3e1fcd2208,-1,-0.036172
2926,56e2083f-9c02-40c8-8af6-cdb4097c3395,-1,-0.036094
7236,d0616afe-8390-42f6-b3ca-77419d047947,-1,-0.033740
2264,43f38430-6700-4215-9eea-c091bd67b5c6,-1,-0.032899


In [8]:
# 5. 计算评估指标：AUC、PR-AUC、Precision、Recall、F1

from sklearn.metrics import roc_auc_score, average_precision_score, precision_score, recall_score, f1_score

# 账户级真实标签：按 card_id 聚合明细标签，这里用 max 规则（账户内只要有一条是 1，就认为账户为 1）
card_label = (
    df.groupby('card_id')['label']
    .max()
    .reindex(result_df['card_id'])  # 按 result_df 对齐
)

# 转成 numpy 数组
y_true = card_label.values.astype(int)

# Isolation Forest 的 iso_score 越小越异常，
# 为了让“越大越可疑”更直观，这里取负号作为异常分数
anomaly_prob = -result_df['iso_score'].values

# AUC-ROC
auc = roc_auc_score(y_true, anomaly_prob)

# PR-AUC（Average Precision）
pr_auc = average_precision_score(y_true, anomaly_prob)

# 二值预测阈值：用 iso_label（-1=异常，1=正常）
y_pred = (result_df['iso_label'].values == -1).astype(int)

precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

metrics = {
    'AUC': auc,
    'PR_AUC': pr_auc,
    'Precision': precision,
    'Recall': recall,
    'F1': f1,
}

metrics

{'AUC': 0.6266452581565872,
 'PR_AUC': 0.2834020459731023,
 'Precision': 0.2774663677130045,
 'Recall': 0.2777777777777778,
 'F1': 0.2776219854178351}